In [1]:
import pandas as pd
import syft as sy
import numpy as np
sy.logger.remove()

Seeding DB with roles!


### Logging into the domain Nodes

In [2]:
# We will login into Canada and Italy domain node
ca = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8081)
it = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8082)

Connecting to http://localhost:8081... done! 	 Logging into canada... done!
Connecting to http://localhost:8082... done! 	 Logging into italy... done!


In [17]:
ca.datasets

,data,id,name,description,tags
0,"[{'name': 'feb2020-100', 'id': '57e00ef9-3073-...",8d974739-2f47-43fe-bb0a-98d7608b8ac4,Canada Trade Data - First 100 rows,A collection of reports from Canada's statisti...,[]
1,"[{'name': 'feb2020-100', 'id': '65f584fa-2a86-...",d003ae2b-917b-4ff2-9920-a68aadd530a1,Canada Trade Data - First 100 rows,A collection of reports from Canada's statisti...,[]


In [18]:
it.datasets

,tags,description,id,data,name
0,[],A collection of reports from iStat's statistic...,deff7e0b-63c9-4eed-92ab-a334d09bbe92,"[{'name': 'feb2020-100', 'id': '47eaf944-6d8b-...",Italy Trade Data - First 100 rows
1,[],A collection of reports from iStat's statistic...,c09e1a0c-8314-4338-8ea0-766739fad841,"[{'name': 'feb2020-100', 'id': 'd648a7a5-7a45-...",Italy Trade Data - First 100 rows


In [19]:
ca.privacy_budget

10.0

In [20]:
it.privacy_budget

10.0

In [21]:
ca_data = ca.datasets[1]['feb2020-100']
it_data = it.datasets[1]['feb2020-100']

In [22]:
ca_data

<TensorWrappedSingleEntityPhiTensorPointer -> canada:65f584fa2a86440782a658ec4f29b32f>

In [24]:
# result = ca_data + it_data

In [25]:
result = ca_data.share(it)

In [41]:
public_result = result.publish(sigma=10)

a
b
c
d
a
b
c
d


In [44]:
public_result.get()

Tensor(child=ShareTensor(child=[[-10   7   0]
 [ 13 -19   4]
 [ -4  -1  -4]
 [-18   8  16]
 [ 17   8 -14]
 [ 10 -15  -7]
 [  1  -5  13]
 [ -5   2   6]
 [  0  16 -13]
 [ 20 -19  30]]))

In [45]:
it.privacy_budget

10.0

In [46]:
ca.privacy_budget

10.0